https://www.nrcan.gc.ca/energy/publications/efficiency/buildings/5985 Table 1

In [1]:
import pandas as pd
import numpy as np

In [236]:
def calculate_energy_intensity(row):
    def zero_nulls(x):
        if pd.isnull(x):
            return 0
        else:
            return x
    
    #calculate the equivalant kilo watt hours
    ekWh  = (1.0)     * zero_nulls(row['Electricity'])
    ekWh += (10.6273) * zero_nulls(row['Natural Gas'])
    ekWh += (10.74)   * zero_nulls(row['Fuel Oil 1 & 2'])
    ekWh += (11.25)   * zero_nulls(row['Fuel Oil 4 & 6'])
    ekWh += (7.09)    * zero_nulls(row['Propane'])
    ekWh += (3876.7)  * zero_nulls(row['Wood'])

    #Facilities related to the treatment of water
    if row['Annual Flow'] > 0:
        ekWh_per_ML = ekWh/row['Annual Flow']
        GJ_per_ML   = ekWh_per_ML * 0.0036  #0.0036 GJ/ekWh
        return GJ_per_ML, ekWh_per_ML

    #Calculation per square footage
    
    sqft = 0
    if row['Unit1'] in ['Square feet','Square Feet','square feet',u'pieds carrés',u'Pieds carrés','Sq Ft']:
        sqft = row['Total Floor Area']
    elif row['Unit1'] in ['Square meters','square meters','Square Meters']:
        sqft = row['Total Floor Area'] * 10.7639
    
    if sqft > 0:
        ekWh_per_sqft = ekWh/sqft
        GJ_per_m2 = ekWh_per_sqft * 0.0036/0.092903 #0.0036 GJ/ekWh, and 0.092903 m2/sqft
        return ekWh_per_sqft, GJ_per_m2
    else:
        return None, None

In [71]:
excel_file = pd.ExcelFile('bps_energy_consumption_2011_2012.xlsx', encoding='utf-8')
raw_2011 = excel_file.parse('2011 Raw')

In [141]:
raw_2011.ix[2367]

Organization Name                                                 Alexandra Hospital
Sector Name                                                          Public Hospital
SubSector Name                                                         Acute/Chronic
Operation Name                                                    Alexandra Hospital
Operation Type                         Administrative offices and related facilities
Address                                                              29 Noxon Street
City                                                                       Ingersoll
ZIP/Postal Code                                                              N5C 3V6
Total Floor Area                                                              3813.3
Unit1                                                                  Square meters
Annual Flow                                                                      NaN
Average Hours Per Week                                           

In [235]:
calculate_energy_intensity(raw_2011.ix[2367])

(74.617655462978234, 2.8914411769988226)

In [237]:
raw_2011.ix[2]

Organization Name                                                  Town of Goderich
Sector Name                                                               Municipal
SubSector Name                                                         Municipality
Operation Name                                                Water Treatment Plant
Operation Type                         Facilities related to the treatment of water
Address                                                                 100 Cove Rd
City                                                                       Goderich
ZIP/Postal Code                                                             N7A 2K5
Total Floor Area                                                                733
Unit1                                                                 Square meters
Annual Flow                                                                1405.631
Average Hours Per Week                                                      

In [238]:
calculate_energy_intensity(raw_2011.ix[2])

(2.5477659499541487, 707.71276387615239)